In [1]:
import os
from datetime import datetime
import time
import dotenv
import pandas as pd

import requests
import requests.auth

import praw

import openai

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# load secrets from .env into environment variables
dotenv.load_dotenv()

praw.__version__

'7.7.0'

todo, make proper readme, 
 - objective is to use OpenAI for named entity extraction to extract all the songs form [this reddit thread](https://www.reddit.com/r/AskReddit/comments/12viv4v/what_is_the_prettiest_song_you_ever_heard_in_your/) and make Spotify playlist
 - use Reddit PRAW API to download all the comments (get [Reddit API key](https://www.reddit.com/prefs/apps))
 - use OpenAI API with a prompt like, extract all the songs from this text to CSV get ([OpenAI API key](https://platform.openai.com/account/api-keys))
 - use Spotify API to make a playlist (get [Spotify API key](https://developer.spotify.com/documentation/web-api/tutorials/getting-started))
 - works, needed a lot of scrubbing, but about 1 day of work, wouldn't have been possible to do a 700-song playlist manually without a team of Mechanical Turks or something
 - If I wanted to go nuts, would process comments individually, save a file for each comment's extracted songs, would make it easier to track down what OpenAI gets wrong, have a resumable, retryable, repeatable process and 
 - output a big github table of all the songs in the readme and link to spotify playlist and individual songs
 
 needs a .env file per dot-env-template
 

## Get all comments from a reddit posting

In [2]:
def getPraw():
    return praw.Reddit(user_agent="prettiest_song/0.001", 
                       client_id=os.getenv('CLIENT_ID'), 
                       client_secret=os.getenv('CLIENT_SECRET'))


def getAll(r, submissionId, verbose=True):
    submission = r.submission(submissionId)
    submission.comments.replace_more(limit=None)
    commentsList=submission.comments.list()
    return commentsList


In [3]:
submission = "12viv4v"
print(datetime.now())
r = getPraw()
res = getAll(r, submission)
print(datetime.now())

print("retrieved ", len(res), 'comments')

2023-04-27 22:03:09.201043
2023-04-27 22:42:13.577094
retrieved  24836 comments


In [5]:
# we have a list of comment objects
# filter comments with at least 5 karma
res3 = [r for r in res if r.score >= 5]
print('filtered to ', len(res3), 'comments')
res3[0].body, res3[0].score

filtered to  2476 comments


('Gymnopédies - Erik Satie', 6886)

## Extract artists and song titles using OpenAI

In [32]:
# check lengths of posts
for i in range(len(res3)):
    if len(res3[i].body) <3:
        print (i, res3[i].body)
    if len(res3[i].body) > 4096:
        print(i, len(res3[i].body))
        

473 4162
1512 W


In [95]:
# avg length
sum([len(r.body) for r in res3]) / len(res3)

106.34289176090469

In [33]:
print (res3[473].body)

Saturn by Sleeping at Last:
https://www.youtube.com/watch?v=dzNvk80XY9s

The version they did with Tim Fain is even more beautiful: 
https://www.youtube.com/watch?v=0nRpeAiur9Q

I'm not good at choosing one thing from a list of favorites as the best, so I've got about 30+ answers that are really a 30+ -way tie, and the one that I would consider as "prettiest" at any given moment is heavily influenced by my current mood. So, it could be any one of these from my "Heart Wrenchingly Beautiful" playlist on Tidal, as listed below.  Hope that some of these songs create that feeling in you as well that the universe contains things that are incomprehensibly beautiful:


* **Title:** Innerbloom
* **Artist:** RÜFÜS DU SOL
* **Album:** Innerbloom (The Remixes)


--------


* **Title:** Hallelujah
* **Artist:** Rufus Wainwright
* **Album:** Vibrate: The Best Of (Deluxe Edition)


--------


* **Title:** In This Shirt
* **Artist:** The Irrepressibles
* **Album:** A Long Way Down - Original Motion Pi

In [36]:
# for each comment object we will extract the body 
# then submit as part of a prompt to chatgpt
openai.api_key = os.getenv('OPENAI_API_KEY')

slist = res3.copy()

maxchars = 5000                  # max tokens is 4096 but we'll limit each prompt to 5000 chars
# make sure no single post > maxchars + prompt
for i in range(len(slist)):
    if len(slist[i].body) <3:
        print (i, slist[i].body)
    if len(slist[i].body) > maxchars:
        print (i, slist[i].body)
        
outfile = open('bronze.txt', 'w')
logdir = 'logs'
count = 0
c = 0

prefix = """Define an example CSV file output as follows: 
"artist","song_title"
"The Beatles","Yesterday"
"Eagles","Hotel California"

Extract all song titles and artists from the following input, and return a CSV file output of the artists and song titles you extract from the input. If there were no songs extracted from the input, return "no songs found". the input is:
"""

while(slist):  # still comments to process
    prompt = ""
    for _ in range(100):  # add up to 100 posts to the prompt
        if slist:
            if len(prompt) + len(slist[0].body) < maxchars:  
                body = slist.pop(0).body # in order, for better context 
#                 if len(body) <3:
#                     print (c, body)
                prompt += body
                prompt += " \n \n"
#                 c += 1            
            
    # retry loop, have received untrapped 502 error
    for _ in range(3):
        try:
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo-0301',
                messages=[{"role":"user", 
                           "content": prefix + prompt}],
                temperature=0,
            )
        except Exception as error:
            print("An exception occurred:", error) 
        break
        print("retrying...")

    outfile.write(response['choices'][0]['message']['content'])
    outfile.write('\n\n')
    outfile.flush()
    
    with open("%s/%04d.log" % (logdir, count), 'w') as logfile:
        logfile.write(prompt)
        logfile.write('\n\n===========\n\n')
        logfile.write(response['choices'][0]['message']['content'])
 
    count += 1
#     print(c)
    print('.', end='')

outfile.close()
print(datetime.now())



1512 W
80
.164
.215
.299
.361
.391
.447
.473
.486
.517
.563
.606
.646
.684
.725
.758
.800
.841
.880
.922
.974
.1020
.1068
.1116
.1169
.1213
.1268
.1327
.1388
.1439
.1493
.1512 W
1550
.1598
.1636
.1676
.1701
.1755
.1790
.1825
.1873
.1915
.1948
.1996
.2032
.2086
.2131
.2171
.2200
.2226
.2256
.2286
.2322
.2364
.2407
.2445
.2474
.2476
.2023-04-28 10:51:18.577363


In [37]:
# will have to tweak the file to get it to load

df = pd.read_csv("bronze.txt")
df


,artist,song_title
0,Don McLean,"Vincent (Starry, Starry Night)"
1,Lord Huron,The night we met
2,Simon & Garfunkel,Scarborough Fair
3,Neil Young,Harvest Moon
4,John Denver,Annie’s Song
...,...,...
1063,Apocalyptica,(unknown)
1064,Nightwish,(unknown)
1065,Output:,NaN
1066,artist,song_title


In [43]:
df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "song_title"]) \
    .to_csv('silver.csv', index=False)

# tweak further to get to gold.csv



## Load into a Spotify playlist


In [38]:
client_credentials_manager = SpotifyClientCredentials(client_id=os.getenv('SPOTIFY_CLIENT_ID'), 
                                                      client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                      )

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [44]:
# check artists
df = pd.read_csv("silver.csv")

dedupe = {}
fail_list = []

for index, artist, title in df.itertuples():
    if artist in dedupe:
        continue
    dedupe[artist]=1
    query_str = 'artist:%s' % (artist)
    artist_results = sp.search(q=query_str, type='artist', limit=3, offset=0, market='US')
    artist_names = [artist['name'] for artist in artist_results['artists']['items']]
    if artist_names:
        if artist.lower() != artist_names[0].lower():
            print(artist, artist_names)
    else:
        fail_list.append((artist, title))
        print("not found:", artist, "-", title)


A-Ha ['Daryl Hall & John Oates', 'a-ha', 'half•alive']
Al Stewart ['Alexander Stewart', 'Al Stewart', 'Alberto Stewart']
America ['The All-American Rejects', 'America', 'American Authors']
not found: Andrew Lloyd Webber, Sarah Brightman, Paul Miles-Kingston - Pie Jesu
Babyface ['Babyface Ray', 'Babyface', 'BabyFaceWood']
Bach ['Johann Sebastian Bach', 'Bachman-Turner Overdrive', 'Tal Bachman']
Beethoven ['Ludwig van Beethoven', 'Camper Van Beethoven', "Beethoven's Wig"]
Beyonce ['Beyoncé', 'Beyonce Smith', 'Mc Beyonce']
Bjork ['Björk', 'Brant Bjork', 'Beatrice Björkman']
Boa ['bôa', 'Boards of Canada', 'BoA']
Cat Stevens ['Yusuf / Cat Stevens', 'Catherine Stevenson', 'Cathy Stevens']
Celine Dion ['Céline Dion', 'Celine Dion (Karaoke)', 'Karaoke - Celine Dion']
not found: Charlie Chaplin, Thomas Beckman - Smile
Chopin ['Frédéric Chopin', 'Chopin Chamber Orchestra', 'Chopin']
Christophe ['Christopher Jackson', 'Christopher Cross', 'Christopher von Uckermann']
Death Cab for Cutie for Cuti

In [74]:
# check songs

df = pd.read_csv("silver.csv")
df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "song_title"])

dedupe = {}
mylist = []
fail_list = []
artist_list, track_list, uri_list, album_list = [], [], [], []
orig_artist, orig_track = [], []

for index, artist, title in df.itertuples():
    query_str = 'artist:%s track:%s' % (artist, title)
    track_results = sp.search(q=query_str, type='track', limit=1, offset=0, market='US')
    results = track_results['tracks']['items']
    
    if results:
        r = results[0]
        # failsafe to never put same track twice
        if dedupe.get(r['id']):
            continue
        dedupe[r['id']]=True
        if title.lower() != r['name'].lower():
            print ("%s|%s : %s|%s" % (artist, title, r['artists'][0]['name'], r['name']))
        uri_list.append(r['uri'])
        artist_list.append(r['artists'][0]['name'])
        track_list.append(r['name'])
        album_list.append(r['album']['name'])
        orig_artist.append(artist)
        orig_track.append(title)
#         print('  ',
#               r['artists'][0]['name'],'|',
#               r['name'], '|',
#               r['album']['name'],'|',
#               r['album']['release_date'],'|',
#               r['popularity'])
    else:
        fail_list.append((artist, title))
        print("not found:", artist, "-", title)

Marvin Berry, The Starlighters|Earth Angel (Will You Be Mine) : Marvin Berry|Earth Angel (Will You Be Mine?)
Nicole Kidman, Ewan McGregor|Come what may : Nicole Kidman|Come What May - From "Moulin Rouge" Soundtrack
Alice In Chains|Don Follow : Alice In Chains|Don't Follow
Bach|Air On G : Johann Sebastian Bach|Orchestral Suite No. 3 in D Major, BWV 1068: II. Air "Air on the G string"
Bach|Brandenburg Concerto : Johann Sebastian Bach|Brandenburg Concerto No. 2 in F Major, BWV 1047: I. Allegro
Bach|Cello Suite No.1 : Johann Sebastian Bach|Cello Suite No. 1 in G Major, BWV 1007: I. Prélude
Bibio|Lover's Carvings : Bibio|lovers’ carvings
Billy Joel|She’s Always A Woman : Billy Joel|She's Always a Woman
Billy Joel|The Nylon Curtain : Billy Joel|Where's the Orchestra
Buddy Holly|Dearest : Buddy Holly|(Ummmm, Oh Yeah) Dearest
Cat Stevens|Moonshadow : Yusuf / Cat Stevens|Moonshadow - Remastered 2021
Cat Stevens|Morning Has Broken : Yusuf / Cat Stevens|Morning Has Broken - Remastered 2021
Cat St

In [75]:
gold_df = pd.DataFrame({'input_artist': orig_artist,
                        'artist': artist_list,
                        'input_track': orig_track,
                        'track': track_list,
                        'album': album_list,
                        'uri': uri_list})
gold_df

,input_artist,artist,input_track,track,album,uri
0,"Ella Fitzgerald, Louis Armstrong",Ella Fitzgerald,Stars Fell On Alabama,Stars Fell On Alabama,The Complete Ella And Louis On Verve,spotify:track:0hGjRgbnFuvOMUKgQK1IGu
1,"Emerson, Lake & Palmer","Emerson, Lake & Palmer",Lucky Man - 2012 Remaster,Lucky Man - 2012 Remaster,"Emerson, Lake & Palmer",spotify:track:6qj0OV5w2PlGTASSx34Lrl
2,"Grover Washington, Jr.","Grover Washington, Jr.",Just the Two of Us (feat. Bill Withers),Just the Two of Us (feat. Bill Withers),Anthology,spotify:track:1ko2lVN0vKGUl9zrU0qSlT
3,"Hans Zimmer, Benjamin Wallfisch",Hans Zimmer,Wallace,Wallace,Blade Runner 2049 (Original Motion Picture Sou...,spotify:track:7cdDskEdvNKYeDr0jaDrey
4,"Marvin Berry, The Starlighters",Marvin Berry,Earth Angel (Will You Be Mine),Earth Angel (Will You Be Mine?),Back To The Future (Original Motion Picture So...,spotify:track:6SJHzGBHrHhrN5nYg2QQdJ
...,...,...,...,...,...,...
674,beabadoobee,beabadoobee,Glue Song,Glue Song,Glue Song,spotify:track:3iBgrkexCzVuPy4O9vx7Mf
675,bôa,bôa,Duvet,Duvet,Twilight,spotify:track:42qNWdLKCI41S4uzfamhFM
676,k.d. lang,k.d. lang,Constant Craving,Constant Craving,Recollection,spotify:track:0wCrg1LhgPcGMw51qqpI6k
677,k.d. lang,k.d. lang,Hallelujah,Hallelujah,Recollection,spotify:track:5S0pF13mOsTnYweFwFy4D0


In [79]:
with pd.option_context("display.max_rows", 999):
    display(gold_df.loc[gold_df['input_artist'].str.lower() != gold_df['artist'].str.lower()])

,input_artist,artist,input_track,track,album,uri
0,"Ella Fitzgerald, Louis Armstrong",Ella Fitzgerald,Stars Fell On Alabama,Stars Fell On Alabama,The Complete Ella And Louis On Verve,spotify:track:0hGjRgbnFuvOMUKgQK1IGu
3,"Hans Zimmer, Benjamin Wallfisch",Hans Zimmer,Wallace,Wallace,Blade Runner 2049 (Original Motion Picture Sou...,spotify:track:7cdDskEdvNKYeDr0jaDrey
4,"Marvin Berry, The Starlighters",Marvin Berry,Earth Angel (Will You Be Mine),Earth Angel (Will You Be Mine?),Back To The Future (Original Motion Picture So...,spotify:track:6SJHzGBHrHhrN5nYg2QQdJ
5,"Nicole Kidman, Ewan McGregor",Nicole Kidman,Come what may,"Come What May - From ""Moulin Rouge"" Soundtrack",Moulin Rouge,spotify:track:5cCAnk1sHJDdqqNIScOkZU
32,America,American Folk Channel,The Boxer,The Boxer,American Country Folk for the Trucks 2,spotify:track:6jLXjtanB0Yp82aAV7VGsN
51,Bach,Johann Sebastian Bach,Air On G,"Orchestral Suite No. 3 in D Major, BWV 1068: I...",Sleep for Baby,spotify:track:77GRzY2dy2IsSpSssSHPjq
52,Bach,Johann Sebastian Bach,Brandenburg Concerto,"Brandenburg Concerto No. 2 in F Major, BWV 104...",Klassische Musik zum Frühstück,spotify:track:19VpImkhO6MH9prYAxytOv
53,Bach,Johann Sebastian Bach,Cello Suite No.1,"Cello Suite No. 1 in G Major, BWV 1007: I. Pré...",Bach: Unaccompanied Cello Suites (Remastered),spotify:track:61dYvvfIRtIDFuqZypPAta
54,Bach,Johann Sebastian Bach,Toccata and Fugue in D minor,Toccata and Fugue in D minor,Great Organ Classics,spotify:track:5cYyCE4b3T5TdQ3sGg8CYc
107,Cat Stevens,Yusuf / Cat Stevens,Moonshadow,Moonshadow - Remastered 2021,Teaser And The Firecat (Remastered 2021),spotify:track:5p3nvmoaAY03qGmBQApn0w


In [81]:
with pd.option_context("display.max_rows", 999):
    display(gold_df.loc[gold_df['input_track'].str.lower() != gold_df['track'].str.lower()])

,input_artist,artist,input_track,track,album,uri
4,"Marvin Berry, The Starlighters",Marvin Berry,Earth Angel (Will You Be Mine),Earth Angel (Will You Be Mine?),Back To The Future (Original Motion Picture So...,spotify:track:6SJHzGBHrHhrN5nYg2QQdJ
5,"Nicole Kidman, Ewan McGregor",Nicole Kidman,Come what may,"Come What May - From ""Moulin Rouge"" Soundtrack",Moulin Rouge,spotify:track:5cCAnk1sHJDdqqNIScOkZU
21,Alice In Chains,Alice In Chains,Don Follow,Don't Follow,Jar Of Flies,spotify:track:7CI0VS6ETno5mgHi4PoCrY
51,Bach,Johann Sebastian Bach,Air On G,"Orchestral Suite No. 3 in D Major, BWV 1068: I...",Sleep for Baby,spotify:track:77GRzY2dy2IsSpSssSHPjq
52,Bach,Johann Sebastian Bach,Brandenburg Concerto,"Brandenburg Concerto No. 2 in F Major, BWV 104...",Klassische Musik zum Frühstück,spotify:track:19VpImkhO6MH9prYAxytOv
53,Bach,Johann Sebastian Bach,Cello Suite No.1,"Cello Suite No. 1 in G Major, BWV 1007: I. Pré...",Bach: Unaccompanied Cello Suites (Remastered),spotify:track:61dYvvfIRtIDFuqZypPAta
67,Bibio,Bibio,Lover's Carvings,lovers’ carvings,Ambivalence Avenue,spotify:track:2cRYvcyWfz7jieYIBkI7JR
76,Billy Joel,Billy Joel,She’s Always A Woman,She's Always a Woman,The Stranger (Legacy Edition),spotify:track:5RgFlk1fcClZd0Y4SGYhqH
77,Billy Joel,Billy Joel,The Nylon Curtain,Where's the Orchestra,The Nylon Curtain,spotify:track:32ca7I3D9UbIUR4hcCE3MH
103,Buddy Holly,Buddy Holly,Dearest,"(Ummmm, Oh Yeah) Dearest",Giant,spotify:track:6NDg4EyQE5lalcCEEpytsM


In [51]:
gold_df.iloc[29]

input_artist                                   America
artist                           American Folk Channel
input_track                                  The Boxer
track                                        The Boxer
album           American Country Folk for the Trucks 2
uri               spotify:track:6jLXjtanB0Yp82aAV7VGsN
Name: 29, dtype: object

In [82]:
# add manually, plus 'not found'
bad_lookups = [
    32,422,556,564,575,621, 622,638,275
]

for i in bad_lookups:
    print(gold_df.iloc[i])

input_artist                                   America
artist                           American Folk Channel
input_track                                  The Boxer
track                                        The Boxer
album           American Country Folk for the Trucks 2
uri               spotify:track:6jLXjtanB0Yp82aAV7VGsN
Name: 32, dtype: object
input_artist                         Michael Jackson
artist                Made famous by Michael Jackson
input_track                 Stop Till You Get Enough
track                  Dont stop till you get enough
album                        Karaoke Michael Jackson
uri             spotify:track:1T6lR1zTxvh4xzEn1M48Gc
Name: 422, dtype: object
input_artist                             The Beatles
artist                  Peel, David & The Apple Band
input_track                               Abbey Road
track              The wonderful world of Abbey Road
album                         Bring Back The Beatles
uri             spotify:track:6zZU0UM3

In [83]:
gold_df = gold_df.drop(
    axis='index',
    labels=bad_lookups)

gold_df[['artist', 'track']].to_csv('gold.csv', index=False)

with pd.option_context("display.max_rows", 999):
    display(gold_df)

,input_artist,artist,input_track,track,album,uri
0,"Ella Fitzgerald, Louis Armstrong",Ella Fitzgerald,Stars Fell On Alabama,Stars Fell On Alabama,The Complete Ella And Louis On Verve,spotify:track:0hGjRgbnFuvOMUKgQK1IGu
1,"Emerson, Lake & Palmer","Emerson, Lake & Palmer",Lucky Man - 2012 Remaster,Lucky Man - 2012 Remaster,"Emerson, Lake & Palmer",spotify:track:6qj0OV5w2PlGTASSx34Lrl
2,"Grover Washington, Jr.","Grover Washington, Jr.",Just the Two of Us (feat. Bill Withers),Just the Two of Us (feat. Bill Withers),Anthology,spotify:track:1ko2lVN0vKGUl9zrU0qSlT
3,"Hans Zimmer, Benjamin Wallfisch",Hans Zimmer,Wallace,Wallace,Blade Runner 2049 (Original Motion Picture Sou...,spotify:track:7cdDskEdvNKYeDr0jaDrey
4,"Marvin Berry, The Starlighters",Marvin Berry,Earth Angel (Will You Be Mine),Earth Angel (Will You Be Mine?),Back To The Future (Original Motion Picture So...,spotify:track:6SJHzGBHrHhrN5nYg2QQdJ
5,"Nicole Kidman, Ewan McGregor",Nicole Kidman,Come what may,"Come What May - From ""Moulin Rouge"" Soundtrack",Moulin Rouge,spotify:track:5cCAnk1sHJDdqqNIScOkZU
6,311,311,Amber,Amber,Greatest Hits '93 - '03,spotify:track:51UtgWS4z1eMPuLQOzPtNH
7,A Perfect Circle,A Perfect Circle,3 Libras,3 Libras,Mer De Noms,spotify:track:5kHkaBN8OEQlmXfQkACxSt
8,A Perfect Circle,A Perfect Circle,Blue,Blue,Thirteenth Step,spotify:track:6tgTTBaIf0tO6lvDhoXfMg
9,A Perfect Circle,A Perfect Circle,Gimme Gimme Gimme,Gimme Gimme Gimme,eMOTIVe,spotify:track:1UxX0RyfqdhM4lpylbNv1W


In [85]:
# get playlist ids
# first create a playlist in UI to load songs
playlists = sp.user_playlists(os.getenv('SPOTIFY_USERNAME'))
while playlists:
    for i, playlist in enumerate(playlists['items']):
        if playlist['name'] != 'Reddit Prettiest Songs':
            continue
        print(playlist['id'])
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

08YFkbtTV6GBfNtjJ4PHDu
   1 spotify:playlist:08YFkbtTV6GBfNtjJ4PHDu Reddit Prettiest Songs


In [90]:
# must follow an oauth workflow to write a playlist in Spotify
# running this cell should request a spotify login and then redirect to an url
# paste whole url with id into form to authenticate

scope = "playlist-modify-public"

sp = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(scope=scope,
                                                       client_id=os.getenv('SPOTIFY_CLIENT_ID'),
                                                       client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                       redirect_uri="https://druce.ai"
                                                      ))

In [91]:
addlist = gold_df['uri'].to_list()
print (len(addlist))

while(addlist):
    sp.user_playlist_add_tracks(os.getenv('SPOTIFY_USERNAME'), 
                                playlist_id='08YFkbtTV6GBfNtjJ4PHDu', 
                                tracks=addlist[-100:])
    addlist = addlist[:-100]
    print("added items, remaining ", len(addlist))


670
Enter the URL you were redirected to: https://druce.ai/?code=AQBguiz-XgLfT1bxpj8Tg8SEYUfZ34OWP4GRzIxYK4zdudcEMixglfE75lLrQEZ79MX5IoafVBxKAAy8nQiN_icmmCmyRuIKBrXO6KcjK8fKPtbqTqNJ8BNOsdgUov-mPpMIg6hqG0QXRzBBIRQhVqJA9Jaj5dj6Fsa8HC2wuysoWkFPLHJXbB9KQ-M
added items, remaining  570
added items, remaining  470
added items, remaining  370
added items, remaining  270
added items, remaining  170
added items, remaining  70
added items, remaining  0


In [ ]:
# manually add the ones that weren't found for some reason
